In [2]:
# !wget https://huggingface.co/datasets/aisyahhrazak/crawl-soalan/resolve/main/soalan-pt3online.jsonl
# !wget https://huggingface.co/datasets/aisyahhrazak/crawl-soalan/resolve/main/soalan-upsr.jsonl
# !wget https://huggingface.co/datasets/aisyahhrazak/crawl-soalan/resolve/main/soalanspm.jsonl
# !wget https://huggingface.co/datasets/aisyahhrazak/crawl-soalan/resolve/main/spm-ayatpasif-aktif.jsonl

In [3]:
import json
from unidecode import unidecode

In [4]:
instructions = []

files = [
    'soalan-pt3online.jsonl',
    'soalan-upsr.jsonl',
    'soalanspm.jsonl',
]

for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            q = unidecode(l['question']).split(') ')[1:]
            q = ') '.join(q).strip()
            a = l['answer'].strip()
            choices = l['answer_choice'].split('\n')
            choices = [c for c in choices if len(c)]
            answer = None
            for c in choices:
                if a in c:
                    answer = c.split('.')[0]
                    break
            c = '\n'.join(choices)
            s = f"Jawab soalan berikut.\n{q}\n\n{c}\nJawapan: {answer}"
            instructions.append(s)

In [5]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai2-aiservices.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [6]:
!mkdir exam

In [8]:
from tqdm import tqdm
import os

for i in tqdm(range(0, len(instructions), 1)):
    filename = f'exam/{i}.json'
    if os.path.exists(filename):
        continue
    try:
        message_text = [
            {"role":"user","content": f"{instructions[i]}\n\ngenerate similar questions included answers like above."},
        ]
        completion = openai.ChatCompletion.create(
          engine="gpt-4",
          messages = message_text,
          temperature=1.0,
          max_tokens=1024,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        with open(filename, 'w') as fopen:
            json.dump(completion.choices[0]['message']['content'], fopen)
    except Exception as e:
        print(e)

 91%|███████████████████████████████████▌   | 651/715 [1:41:19<09:29,  8.90s/it]

'content'


100%|███████████████████████████████████████| 715/715 [1:52:06<00:00,  9.41s/it]


In [12]:
augmentation = []
for i in tqdm(range(0, len(instructions), 1)):
    filename = f'exam/{i}.json'
    try:
        with open(filename) as fopen:
            data = json.load(fopen)
        splitted = data.split('\n\n')
        for s in splitted:
            a = s.split('Jawapan:')[1].strip()
            augmentation.append({
                'question': s.split('Jawapan:')[0].strip(),
                'answer': a,
                
            })
    except:
        pass

100%|██████████████████████████████████████| 715/715 [00:00<00:00, 31377.41it/s]


In [13]:
len(augmentation)

3612

In [14]:
with open('synthetic-exam.jsonl', 'w') as fopen:
    for a in augmentation:
        fopen.write(f'{json.dumps(a)}\n')